## Needleman-Wunsch

Needleman-Wunsch is the most basic global alignment algorithm. It uses a dynamic programming approach to find all alignments between two sequences which are optimal with respect to some pre-determined scores for matches, mismatches, and gaps. Needleman-Wunsch attempts to maximize the alignment score by efficiently testing all possible alignments (in so far as testing all possible alignments can be efficient, in this case $O(n^3)$).

The basic version originally described by its authors uses the simple scoring technique described above. More advanced techniques use a mismatch matrix to assign different scores depending on which residues are being compared; for example, we might want a mismatch between G and C to have a lower penalty than between G and A. The simple implementation below opts to use the original naive system.

In [25]:
DEFAULT_GAP = -6
DEFAULT_MATCH = 5
DEFAULT_MISMATCH = -2

In [54]:
def makeTable(X, Y, g):
    ''' Given 2 sequences and the gap penalty, construct the Needleman-Wunsch starting table.
    
    Args:
        X (string): The first sequence.
        Y (string): The second sequence.
        g (int): The gap penalty.
    
    Returns:
        list: A len(X)+1 by len(Y)+1 table where all positions are 0 except the first row and column,
        which are filled linearly with the gap penalty.
    
    '''
    table = []
    for i in range(0, len(X)+1):
        table.append([0]*(len(Y)+1))
    for i in range(0,len(X)+1,1):
        table[i][0] = i*g
    for j in range(0,len(Y)+1,1):
        table[0][j] = j*g

    return table


def printTable(table, X, Y):
    ''' Print out a scoring table and its two sequences in a nice format.
    
    Args:
        table (list): The Needleman-Wunsch scoring table.
        X (string): The first sequence, with sentinal character.
        Y (string): The second sequence, with sentinal character.
    '''
    pt = ' '

    for y in Y:
        pt += y.rjust(4)
    pt += '\n'
    for i in range(0, len(table)):
        prow = X[i]
        for c in table[i]:
            prow += str(c).rjust(4)
        prow += '\n'
        pt += prow
    print pt


def traceBack(table, X, Y, g, m, n, i, j, a1, a2, results):
    ''' Recursively traceback a scoring table to construct the alignments.
    
    Args:
        table (list): The completed scoring table.
        X (string): The first sequence, with sentinal.
        Y (string): The second sequence, with sentinal.
        g (int): The gap penalty.
        m (int): The match score.
        n (int): The mismatch penalty.
        i (int): The index in X.
        j (int): The index in Y.
        a1 (str): The alignment string for X.
        a2 (str): The alignment string for Y.
        results (list): List object to place results in.
    '''
    
    if i < 0 or j < 0:
        raise IndexError('Tracback went out of bounds ({i},{j})'.format(i=i, j=j))
    if i == 0 and j != 0:
        traceBack(table, X, Y, g, m, n, i, j-1, a1+'-', a2+Y[j], results)
    if j == 0 and i != 0:
        traceBack(table, X, Y, g, m, n, i-1, j, a1+X[i], a2+'-', results)
    if i == 0 and j == 0:
        a1 = a1[::-1]
        a2 = a2[::-1]
        results.append((a1, a2))
    else:
        a = table[i][j]
        b = table[i-1][j]
        c = table[i][j-1]
        d = table[i-1][j-1]
        if a-d == m and X[i] == Y[j]:
            traceBack(table, X, Y, g, m, n, i-1, j-1, a1+X[i], a2+Y[j], results)
        if a-d == n and X[i] != Y[j]:
            traceBack(table, X, Y, g, m, n, i-1, j-1, a1+X[i], a2+Y[j], results)
        if a - b == g:
            traceBack(table, X, Y, g, m, n, i-1, j, a1+X[i], a2+'-', results)
        if a - c == g:
            traceBack(table, X, Y, g, m, n, i, j-1, a1+'-', a2+Y[j], results)

In [58]:
def NeedlemanWunsch(X, Y, gap, match, mismatch, quiet=True):
    ''' Perform a global alignment between the sequences X and Y using the Needleman-Wunsch algorithm.
    
    Args:
        X (str): The first sequence to be aligned.
        Y (str): The second sequence to be aligned.
        gap (int): The gap penalty.
        match (int): The match score.
        mismatch (int): The mismatch penalty.
        quiet (Optional[bool]): Whether or not to print out diagnostic information. Defaults to True.
        
    Returns:
        list: A list of the optimal alignments.
    '''
    table = makeTable(X, Y, gap)
    
    if not quiet:
        print "===Forward Needleman-Wunsch==="
        print "Table Initialized"
        
    X = '*' + X
    Y = '*' + Y
    
    if not quiet:
        printTable(table, X, Y)
    
    score = lambda a, b: match if a == b else mismatch

    for i in range(1, len(table)):
        for j in range(1, len(table[i])):
            a = table[i-1][j-1] + score(X[i], Y[j])
            b = table[i-1][j] + gap
            c = table[i][j-1] + gap
            table[i][j] = max(a, b, c)
    if not quiet:
        print "Completed Table"
        printTable(table, X, Y)

        print "Optimal Alignments:\n"

    results = []
    traceBack(table, X, Y, gap, match, mismatch, len(X)-2, len(Y)-2, '', '', results)
    if not quiet:
        for a1, a2 in results:
            print a1, '\n', a2, '\n'
        print "--End of Optimal Alignment List--"

    if not quiet:
        print "Parameters: gap={} match={} mismatch={}".format(gap, match, mismatch)
        print "Optimal Global Alignment Score: {}".format(table[len(table)-1][len(table[0])-1])
        print "=" * 60 + '\n'
    
    return results

In [59]:
NeedlemanWunsch('ATTCGGCT', 'AGTTGGGCCCGCGT', DEFAULT_GAP, DEFAULT_MATCH, DEFAULT_MISMATCH, quiet=True)

[('A-TT----CGGC-', 'AGTTGGGCCCGCG'),
 ('A-TT---C-GGC-', 'AGTTGGGCCCGCG'),
 ('A-TT---CG-GC-', 'AGTTGGGCCCGCG'),
 ('A-TT-CG---GC-', 'AGTTGGGCCCGCG'),
 ('A-TTC-G---GC-', 'AGTTGGGCCCGCG'),
 ('A-TTCG----GC-', 'AGTTGGGCCCGCG'),
 ('A-TTCGG----C-', 'AGTTGGGCCCGCG'),
 ('A-TTCGG--C---', 'AGTTGGGCCCGCG'),
 ('A-TTCGG-C----', 'AGTTGGGCCCGCG'),
 ('A-TTCGGC-----', 'AGTTGGGCCCGCG')]